# 08: A2A Orchestration: The Autonomous Pipeline 🤖

This notebook demonstrates the **Grand Orchestration** of the SalesOps Agent Suite (Day 7).

Previously, we manually ran each agent in separate notebooks. Now, we introduce the **`A2ACoordinator`**, a master agent that autonomously manages the entire lifecycle:
1.  **Ingest** raw data.
2.  **Detect** statistical anomalies.
3.  **Explain** outliers using parallel AI workers.
4.  **Act** by triggering enterprise workflows (Jira/Email).

### 🎯 Goals
1.  **Start Environment:** Ensure the Mock Enterprise Server is running on port 7777.
2.  **Execute Pipeline:** Trigger a single `run_pipeline()` command that fans out work to sub-agents.
3.  **Verify Autonomy:** Confirm that data flows seamlessly from CSV to Jira Ticket without human intervention.
4.  **Inspect Artifacts:** Review the generated `manifest.json` and audit logs.

### 🏗️ Components Used
* `agents.orchestrator.A2ACoordinator`: The central brain implementing the Sequential & Parallel agent patterns.
* `agents.*`: All previously built agents working in concert.

## 1) Imports

In [1]:
import sys
import os
import time
import requests
import json
from subprocess import Popen
from dotenv import load_dotenv

# 1. Environment Setup
# Add project root to path so we can import agents
project_root = os.path.abspath(os.path.join(os.path.dirname("__file__"), ".."))
if project_root not in sys.path:
    sys.path.append(project_root)

load_dotenv()

from agents.a2a_coordinator import A2ACoordinator

# 2. Ensure Mock Server is Running
print("🔍 Checking Mock Server Status...")
try:
    requests.get("http://localhost:7777/health")
    print("✅ Mock Server is already running.")
except:
    print("🚀 Starting Mock Server (Port 7777)...")
    log_file = open("../outputs/mock_server_orchestrator.log", "w")

    # Start process using the current python executable
    process = Popen(
        [sys.executable, "-m", "uvicorn", "tools.mock_server:app", "--port", "7777"],
        stdout=log_file,
        stderr=log_file,
        cwd=project_root,
    )
    time.sleep(5)  # Wait for startup

    try:
        requests.get("http://localhost:7777/health")
        print("✅ Mock Server Started Successfully.")
    except:
        print("❌ Failed to start Mock Server. Check logs.")

c:\Users\YASH\anaconda3\envs\salesops\Lib\site-packages\google\cloud\aiplatform\models.py:52: FutureWarning: Support for google-cloud-storage < 3.0.0 will be removed in a future version of google-cloud-aiplatform. Please upgrade to google-cloud-storage >= 3.0.0.
  from google.cloud.aiplatform.utils import gcs_utils
2025-11-24 21:41:08,623 - google_adk.google.adk.models.registry - INFO - Updating LLM class for gemini-.* from <class 'google.adk.models.google_llm.Gemini'> to <class 'google.adk.models.google_llm.Gemini'>
2025-11-24 21:41:08,625 - google_adk.google.adk.models.registry - INFO - Updating LLM class for projects\/.+\/locations\/.+\/endpoints\/.+ from <class 'google.adk.models.google_llm.Gemini'> to <class 'google.adk.models.google_llm.Gemini'>
2025-11-24 21:41:08,625 - google_adk.google.adk.models.registry - INFO - Updating LLM class for projects\/.+\/locations\/.+\/publishers\/google\/models\/gemini.+ from <class 'google.adk.models.google_llm.Gemini'> to <class 'google.adk.mod

🔍 Checking Mock Server Status...
🚀 Starting Mock Server (Port 7777)...
✅ Mock Server Started Successfully.


## 2) Define Flow

In [2]:
# 1. Define Flow Config (Sequential Pipeline)
flow_config = {
    "id": "daily_full_run",
    "confirm_actions": True,  # Allow side-effects (sending tickets)
    "parallelism": 3,  # Fan-out for 3 explainers
}

inputs = {"csv_path": "../data/raw/superstore.csv"}

# Unique session for this demo run
session_id = "session:demo-user-01"

print("✅ Flow Configuration Ready:")
print(json.dumps(flow_config, indent=2))

✅ Flow Configuration Ready:
{
  "id": "daily_full_run",
  "confirm_actions": true,
  "parallelism": 3
}


## 3) Run Coordinator

In [3]:
# 2. Execute the Pipeline
coordinator = A2ACoordinator()
print(f"▶️ Starting Run... (Session: {session_id})")

manifest = coordinator.run(flow_config, inputs, session_id)

print(f"\n✅ Run ID: {manifest['run_id']}")
print(f"Status: {manifest['status']}")

2025-11-24 21:41:31,814 - A2ACoordinator - INFO - Starting Run run_20251124T161131Z_96ecd4
2025-11-24 21:41:31,816 - agents.data_ingestor - INFO - Attempting read with encoding='utf-8'...
2025-11-24 21:41:31,841 - agents.data_ingestor - WARNING - Encoding 'utf-8' failed. Retrying...
2025-11-24 21:41:31,843 - agents.data_ingestor - INFO - Attempting read with encoding='latin1'...
2025-11-24 21:41:31,887 - agents.data_ingestor - INFO - Success! Read 9994 rows.
2025-11-24 21:41:31,907 - agents.data_ingestor - INFO - Schema validation passed.
2025-11-24 21:41:32,006 - agents.data_ingestor - INFO - Snapshot saved successfully to D:\01. Github\salesops-suite\outputs\runs\run_20251124T161131Z_96ecd4\snapshot.parquet


▶️ Starting Run... (Session: session:demo-user-01)


2025-11-24 21:41:32,291 - agents.anomaly_stats_agent - INFO - Running Global Z-Score Detector on Sales (w=30, t=3.0)
2025-11-24 21:41:32,300 - agents.anomaly_stats_agent - INFO - Running Grouped IQR Detector on Region (w=14, k=1.5)
2025-11-24 21:41:32,338 - agents.anomaly_stats_agent - INFO - Saved 253 anomalies to D:\01. Github\salesops-suite\outputs\runs\run_20251124T161131Z_96ecd4\anomalies.json
2025-11-24 21:41:33,499 - google_genai.models - INFO - AFC is enabled with max remote calls: 10.
2025-11-24 21:41:33,501 - google_genai.models - INFO - AFC is enabled with max remote calls: 10.
2025-11-24 21:41:33,501 - google_genai.models - INFO - AFC is enabled with max remote calls: 10.
2025-11-24 21:41:35,633 - httpx - INFO - HTTP Request: POST https://generativelanguage.googleapis.com/v1beta/models/gemini-2.5-flash-lite:generateContent "HTTP/1.1 200 OK"
2025-11-24 21:41:35,638 - google_genai.models - INFO - AFC is enabled with max remote calls: 10.
2025-11-24 21:41:35,826 - google_genai


✅ Run ID: run_20251124T161131Z_96ecd4
Status: completed


## 4) Tail Observability

In [4]:
# 3. Inspect the Master Run Manifest
log_path = "../outputs/observability/a2a_runs.jsonl"
print(f"--- Latest Entry in {log_path} ---")

with open(log_path, "r") as f:
    lines = f.readlines()
    if lines:
        last_run = json.loads(lines[-1])
        print(f"Run ID: {last_run['run_id']}")
        print(f"Tasks: {[t['task_id'] for t in last_run['tasks']]}")
        print(f"Artifacts: {json.dumps(last_run['artifacts'], indent=2)}")

--- Latest Entry in ../outputs/observability/a2a_runs.jsonl ---
Run ID: run_20251124T161131Z_96ecd4
Tasks: ['Ingestor', 'Detector', 'Explainer', 'Actor']
Artifacts: {
  "snapshot": "D:\\01. Github\\salesops-suite\\outputs\\runs\\run_20251124T161131Z_96ecd4\\snapshot.parquet",
  "anomalies": "D:\\01. Github\\salesops-suite\\outputs\\runs\\run_20251124T161131Z_96ecd4\\anomalies.json",
  "explanations": "D:\\01. Github\\salesops-suite\\outputs\\runs\\run_20251124T161131Z_96ecd4\\enriched_anomalies.json",
  "actions_log": "D:\\01. Github\\salesops-suite\\outputs\\runs\\run_20251124T161131Z_96ecd4\\actions.jsonl"
}


## 5) Inspect Outputs (Validation)

In [5]:
# 4. Load Generated Artifacts
explanations_path = "../outputs/runs/" + manifest["run_id"] + "/enriched_anomalies.json"
actions_path = "../outputs/actions/actions.jsonl"

print("\n--- 🧠 Generated Explanations (Top 1) ---")
if os.path.exists(explanations_path):
    with open(explanations_path, "r") as f:
        data = json.load(f)
        if data:
            print(f"Anomaly: {data[0].get('anomaly_id')}")
            print(f"Explanation: {data[0].get('explanation_short')}")

print("\n--- ⚡ Executed Actions (Tail) ---")
if os.path.exists(actions_path):
    with open(actions_path, "r") as f:
        last_action = json.loads(f.readlines()[-1])
        print(f"Action Type: {last_action['type']}")
        print(
            f"Result: {last_action['result']['status']} (HTTP {last_action['result'].get('http_code')})"
        )


--- 🧠 Generated Explanations (Top 1) ---
Anomaly: iqr_South_2014-03-18_s24
Explanation: South region sales of 23,661.23 significantly exceed the expected value of 1,159.38, indicated by a high anomaly score of 24.21.

--- ⚡ Executed Actions (Tail) ---
Action Type: create_ticket
Result: success (HTTP 201)


## 6) Error Demo (Chaos Engineering)

In [6]:
# 5. Simulate a Component Failure
print("💥 Enabling Chaos Monkey (Simulating 500 Errors)...")
requests.post(
    "http://localhost:7777/admin/chaos", json={"enabled": True, "failure_rate": 1.0}
)

# Re-run the Action phase logic specifically to see failure handling
# (Using the same coordinator instance to simulate a retry)
print("▶️ Re-attempting Actions with Chaos...")
try:
    # We manually trigger just the action step for the demo speed
    # In a real A2A run, the coordinator would handle this
    with open(explanations_path, "r") as f:
        anoms = json.load(f)[:1]  # Just 1 for speed

    # This should fail/retry and log errors
    res = coordinator.actor.run_batch(anoms)
    print(
        f"Result with Chaos: {res[0]['result']['status']} (Reason: {res[0]['result'].get('reason')})"
    )

except Exception as e:
    print(f"Caught Expected Error: {e}")

# Reset Chaos
requests.post("http://localhost:7777/admin/chaos", json={"enabled": False})
print("✅ Chaos Disabled.")

💥 Enabling Chaos Monkey (Simulating 500 Errors)...
▶️ Re-attempting Actions with Chaos...


2025-11-24 21:42:25,921 - agents.action_agent - WARNING - Server Error 500. Retrying...
2025-11-24 21:42:29,173 - agents.action_agent - WARNING - Server Error 500. Retrying...
2025-11-24 21:42:33,482 - agents.action_agent - WARNING - Server Error 500. Retrying...
2025-11-24 21:42:39,548 - agents.action_agent - WARNING - Server Error 500. Retrying...
2025-11-24 21:42:42,840 - agents.action_agent - WARNING - Server Error 500. Retrying...
2025-11-24 21:42:47,012 - agents.action_agent - WARNING - Server Error 500. Retrying...


Caught Expected Error: 'result'
✅ Chaos Disabled.


## 7) Idempotency Demo

In [7]:
# 6. Idempotency Verify
print("🔄 Re-running the EXACT same flow (Idempotency Check)...")

# Rerun the coordinator with the SAME session and inputs
manifest_2 = coordinator.run(flow_config, inputs, session_id)

# Check the Action Log: We should NOT see new tickets created, but 'success' results from replay
print(f"\nRun 2 Status: {manifest_2['status']}")

# Verify in Audit Log (check for 'Replay' or same IDs)
# Note: Our Mock Server logs 'Replay' in its console, and returns the SAME ticket_id.
# We can verify by checking if ticket IDs are identical if we stored them,
# or relying on the Server logs/audit trail showing no new resource creation.
print("✅ Idempotency Run Complete. Check server logs for 'Replay' confirmation.")

2025-11-24 21:42:59,198 - A2ACoordinator - INFO - Starting Run run_20251124T161131Z_96ecd4
2025-11-24 21:42:59,198 - agents.data_ingestor - INFO - Attempting read with encoding='utf-8'...
2025-11-24 21:42:59,223 - agents.data_ingestor - WARNING - Encoding 'utf-8' failed. Retrying...
2025-11-24 21:42:59,224 - agents.data_ingestor - INFO - Attempting read with encoding='latin1'...
2025-11-24 21:42:59,250 - agents.data_ingestor - INFO - Success! Read 9994 rows.
2025-11-24 21:42:59,261 - agents.data_ingestor - INFO - Schema validation passed.
2025-11-24 21:42:59,288 - agents.data_ingestor - INFO - Snapshot saved successfully to D:\01. Github\salesops-suite\outputs\runs\run_20251124T161131Z_96ecd4\snapshot.parquet
2025-11-24 21:42:59,324 - agents.anomaly_stats_agent - INFO - Running Global Z-Score Detector on Sales (w=30, t=3.0)
2025-11-24 21:42:59,329 - agents.anomaly_stats_agent - INFO - Running Grouped IQR Detector on Region (w=14, k=1.5)
2025-11-24 21:42:59,361 - agents.anomaly_stats_ag

🔄 Re-running the EXACT same flow (Idempotency Check)...


2025-11-24 21:43:00,384 - google_genai.models - INFO - AFC is enabled with max remote calls: 10.
2025-11-24 21:43:00,385 - google_genai.models - INFO - AFC is enabled with max remote calls: 10.
2025-11-24 21:43:00,385 - google_genai.models - INFO - AFC is enabled with max remote calls: 10.
2025-11-24 21:43:02,573 - httpx - INFO - HTTP Request: POST https://generativelanguage.googleapis.com/v1beta/models/gemini-2.5-flash-lite:generateContent "HTTP/1.1 200 OK"
2025-11-24 21:43:02,695 - httpx - INFO - HTTP Request: POST https://generativelanguage.googleapis.com/v1beta/models/gemini-2.5-flash-lite:generateContent "HTTP/1.1 200 OK"
2025-11-24 21:43:02,695 - httpx - INFO - HTTP Request: POST https://generativelanguage.googleapis.com/v1beta/models/gemini-2.5-flash-lite:generateContent "HTTP/1.1 200 OK"
2025-11-24 21:43:03,580 - google_genai.models - INFO - AFC is enabled with max remote calls: 10.
2025-11-24 21:43:03,709 - google_genai.models - INFO - AFC is enabled with max remote calls: 10.


Run 2 Status: completed
✅ Idempotency Run Complete. Check server logs for 'Replay' confirmation.


## ⏭️ Next Step: The "Learning" Agent (Memory Bank)

Success! We have built a fully autonomous pipeline.
1.  **Ingest** → **Detect** → **Explain** → **Act**.
2.  The Coordinator handled errors, retries, and parallel fan-out.
3.  We have a complete audit trail in `../outputs/observability`.

**However, our agent is "Amnesic".**
If the *same* anomaly happens next month, the agent will re-analyze it from scratch. It won't remember: *"Oh, we saw this last time, and the fix was to email the vendor."*

In **Day 8**, we will build the **Memory Bank**.
We will give our agents **Long-Term Semantic Memory** so they can:
* **Recall** past anomalies.
* **Retrieve** successful actions from history.
* **Learn** from previous resolutions.

👉 **Proceed to `notebooks/09_memory_learning.ipynb`.**